Автор материала: Зраев Артем.

Можно использовать в каких угодно целях.

<b> В задании нужно загрузить датасет с данными оттока и ответить на несколько вопросов (написать код). При этом сам датасет уже есть и его необязательно качать с репозитория</b>

Цель задания: проверить базовые навыки работы студентов с Pandas, умение проводить такой же базовый EDA (exploratory data analysis), делать feature engineering и обучать и валидировать модель.

Список столбцов с типами данных в датасете:

- customerID           object
- gender               object
- SeniorCitizen         int64
- Partner              object
- Dependents           object
- tenure                int64
- PhoneService         object
- MultipleLines        object
- InternetService      object
- OnlineSecurity       object
- OnlineBackup         object
- DeviceProtection     object
- TechSupport          object
- StreamingTV          object
- StreamingMovies      object
- Contract             object
- PaperlessBilling     object
- PaymentMethod        object
- MonthlyCharges      float64
- TotalCharges         object
- Churn                object

In [147]:
import pandas as pd
import numpy as np

df = pd.read_csv("C://Users//adminctf//STUDY//L1//WA_Fn-UseC_-Telco-Customer-Churn.csv")
df.head(3)

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes


In [148]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


##### 1. Какое соотношение мужчин и женщин в представленном наборе данных?

In [149]:
df['gender'].value_counts(normalize = True)

Male      0.504756
Female    0.495244
Name: gender, dtype: float64

##### 2. Какое количество уникальных значений у поля InternetService?

In [150]:
len(df['InternetService'].unique())

3

##### 3. Выведите статистики по полю TotalCharges (median, mean, std).

In [151]:
df['TotalCharges'].mask(df['TotalCharges'] == ' ', df['TotalCharges'].replace(' ', '0'), inplace=True)
df['TotalCharges'] = df['TotalCharges'].astype('float64')
print(f"{df['TotalCharges'].describe()} \n+ median = {df['TotalCharges'].median()}")

count    7043.000000
mean     2279.734304
std      2266.794470
min         0.000000
25%       398.550000
50%      1394.550000
75%      3786.600000
max      8684.800000
Name: TotalCharges, dtype: float64 
+ median = 1394.55


В чем странность того, что вы получили? (подсказка: смотреть нужно на тип данных)

##### 4. Сделайте замену значений поля PhoneService  на числовые (Yes->1, No->0)

In [152]:
df['PhoneService'].loc[df['PhoneService'] == 'Yes'] = 1
df['PhoneService'].loc[df['PhoneService'] == 'No'] = 0
df['PhoneService'] = df['PhoneService'].astype('int64')

C:\Users\adminctf\anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [103]:
df['PhoneService'].value_counts()

1    6361
0     682
Name: PhoneService, dtype: int64

##### 5. Сделайте замену пробелов в поле TotalCharges на np.nan и приведите поле к типу данных float32. Затем заполните оставшиеся пропуски значением 0 с помощью метода fillna у столбца. Снова выведите статистики и сравните с тем, что вы видели в вопросе 3

In [24]:
#сразу сделала, только float64. и не получала оставшихся пропусков

##### 6. Сделайте замену значений поля Churn на числовые (Yes -> 1, No - 0)

In [105]:
df['Churn'].loc[df['Churn'] == 'Yes'] = 1
df['Churn'].loc[df['Churn'] == 'No'] = 0
df['Churn'] = df['Churn'].astype('int64')
df['Churn'].value_counts()

0    5174
1    1869
Name: Churn, dtype: int64

##### 7. Сделайте замену значений полей StreamingMovies, StreamingTV, TechSupport  на числовые (Yes -> 1, No -> 0, No internet service->0)

In [106]:
cols = ['StreamingMovies', 'StreamingTV', 'TechSupport']
for column in cols:
    df[column].loc[df[column] == 'Yes'] = 1
    df[column].loc[df[column] != 1] = 0
    df[column] = df[column].astype('int64')
    print(column,'\n',df[column].value_counts())

StreamingMovies 
 0    4311
1    2732
Name: StreamingMovies, dtype: int64
StreamingTV 
 0    4336
1    2707
Name: StreamingTV, dtype: int64
TechSupport 
 0    4999
1    2044
Name: TechSupport, dtype: int64


##### 8. Заполните пропуски в поле PhoneService значением 0

In [107]:
df['PhoneService'].fillna(0, inplace=True)

#### 8. Для нашего датасета оставьте только указанный ниже список полей, удалив все другие и выведите верхние 3 строки

In [109]:
columns = ['gender', 'tenure', 'PhoneService', 'TotalCharges', 
           'StreamingMovies', 'StreamingTV', 'TechSupport', 'Churn']
df = df[columns]
df.head(3)

,gender,tenure,PhoneService,TotalCharges,StreamingMovies,StreamingTV,TechSupport,Churn
0,Female,1,0,29.85,0,0,0,0
1,Male,34,1,1889.50,0,0,0,0
2,Male,2,1,108.15,0,0,0,1


In [110]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   gender           7043 non-null   object 
 1   tenure           7043 non-null   int64  
 2   PhoneService     7043 non-null   int64  
 3   TotalCharges     7043 non-null   float64
 4   StreamingMovies  7043 non-null   int64  
 5   StreamingTV      7043 non-null   int64  
 6   TechSupport      7043 non-null   int64  
 7   Churn            7043 non-null   int64  
dtypes: float64(1), int64(6), object(1)
memory usage: 440.3+ KB


##### 9. Разделите датасет на тренировочную и тестовую выборку (подсказка - воспользуйтесь train_test_split из sklearn.model_selection. Ссылка - https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)

In [111]:
from sklearn.model_selection import train_test_split

features = ['gender', 'tenure', 'PhoneService', 'TotalCharges', 'StreamingMovies', 'StreamingTV', 'TechSupport']
target = 'Churn'
X_train, X_test, y_train, y_test = train_test_split(df[features], df[target])

##### 10. соберите pipeline для поля gender (нужно разобраться и изучить https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html ) из классов ColumnSelector и OHEEncoder, которые уже написаны ниже заранее

In [112]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline

class ColumnSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.key]
    
class NumberSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on numeric columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]
    
class OHEEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        self.columns = []

    def fit(self, X, y=None):
        self.columns = [col for col in pd.get_dummies(X, prefix=self.key).columns]
        return self

    def transform(self, X):
        X = pd.get_dummies(X, prefix=self.key)
        test_columns = [col for col in X.columns]
        for col_ in test_columns:
            if col_ not in self.columns:
                X[col_] = 0
        return X[self.columns]

gender = Pipeline([
                ('selector', ColumnSelector(key='gender')),
                ('ohe', OHEEncoder(key='gender'))
            ])

##### 11. Вызовите метод fit_transform у пайплайна gender и передайте туда нашу тренировочную выборку (пример по ссылке из документации https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html#sklearn.pipeline.Pipeline.fit)

In [113]:
gender.fit_transform(X_train)

,gender_Female,gender_Male
5102,1,0
2606,1,0
6636,1,0
867,1,0
4106,1,0
...,...,...
438,0,1
1195,1,0
1862,1,0
6073,1,0


##### 12. Здесь код писать уже не нужно (все сделано за вас). К полю tenure применяем StandardScaler (нормируем и центрируем). Ссылка - https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html
Вопрос - в каких случаях это может быть полезно?

In [114]:
from sklearn.preprocessing import StandardScaler

tenure =  Pipeline([
                ('selector', NumberSelector(key='tenure')),
                ('standard', StandardScaler())
            ])

##### 13. Напишите аналогичный (как для tenure) преобразователь поля TotalCharges

In [115]:
TotalCharges =  Pipeline([
                ('selector', NumberSelector(key='TotalCharges')),
                ('standard', StandardScaler())
            ])


Объединение всех "кубиков" очень легко сделать таким образом

In [116]:
from sklearn.pipeline import FeatureUnion

number_features = Pipeline([
                ('selector', ColumnSelector(key=['PhoneService',
                                                 'StreamingMovies', 'StreamingTV', 
                                                 'TechSupport']))
            ])

In [117]:
feats = FeatureUnion([('tenure', tenure),
                      ('TotalCharges', TotalCharges),
                      ('continuos_features', number_features),
                      ('gender', gender)], verbose = True)
feature_processing = Pipeline([('feats', feats)])

На этом этапе что мы сделали:
1. написали преобразователь поля gender, который делает OHE кодирование
2. написали преобразователь для поля tenure, который нормирует и центрирует его 
3. повторили п. 2 для поля TotalCharges
3. для всех остальных просто взяли признаки как они есть, без изменений

У нас уже готов наш пайплайн, который преобразовывает признаки. Давайте обучим модель поверх него. В качестве модели возьмем RandomForestClassifier

In [118]:
from sklearn.ensemble import RandomForestClassifier

pipeline = Pipeline([
    ('features',feats),
    ('classifier', RandomForestClassifier(random_state = 42)),
], verbose = True)

In [119]:
X_train.head()

,gender,tenure,PhoneService,TotalCharges,StreamingMovies,StreamingTV,TechSupport
5102,Female,72,1,7881.20,1,1,1
2606,Female,31,1,3205.60,1,1,0
6636,Female,10,1,676.15,0,1,0
867,Female,5,1,104.30,0,0,0
4106,Female,24,1,2264.05,1,1,0


In [120]:
pipeline.fit(X_train, y_train)

[FeatureUnion] ........ (step 1 of 4) Processing tenure, total=   0.0s
[FeatureUnion] .. (step 2 of 4) Processing TotalCharges, total=   0.0s
[FeatureUnion]  (step 3 of 4) Processing continuos_features, total=   0.0s
[FeatureUnion] ........ (step 4 of 4) Processing gender, total=   0.0s
[Pipeline] .......... (step 1 of 2) Processing features, total=   0.0s
[Pipeline] ........ (step 2 of 2) Processing classifier, total=   0.4s


Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('tenure',
                                                 Pipeline(steps=[('selector',
                                                                  NumberSelector(key='tenure')),
                                                                 ('standard',
                                                                  StandardScaler())])),
                                                ('TotalCharges',
                                                 Pipeline(steps=[('selector',
                                                                  NumberSelector(key='TotalCharges')),
                                                                 ('standard',
                                                                  StandardScaler())])),
                                                ('continuos_features',
                                                 Pipeline(steps=[('selector',
          

##### 14. Сделайте прогноз вероятности оттока для X_test с помощью нашего предобученного на предыдущем шаге пайплайна и убедитесь что вам возвращаются вероятности для 2 классов

In [141]:
probs = pipeline.predict_proba(X_test)
probs

array([[1.  , 0.  ],
       [0.91, 0.09],
       [0.59, 0.41],
       ...,
       [0.97, 0.03],
       [0.29, 0.71],
       [0.99, 0.01]])

##### 15. Посчитайте метрики качества получившейся модели (roc_auc, logloss)

In [144]:
from sklearn.metrics import roc_auc_score, log_loss
roc_auc = roc_auc_score(y_test, pipeline.predict(X_test))
logloss = log_loss(y_test, probs)
print(f'roc_auc = {roc_auc}\nlogloss = {logloss}')

roc_auc = 0.6625270192555575
logloss = 0.760542694949583


### Сохраним наш пайплайн

In [146]:
!pip install dill
import dill
with open("C://Users//adminctf//STUDY//L1//model_RF.dill", "wb") as f:
    dill.dump(pipeline, f)